# Extreme annotators analysis

__Objective:__ on the Kumar dataset, identify the IDs of the extreme annotators, e.g. the ones consistently annotating as non-toxic texts that are _mostly_ considered toxic (according to some criterion).

In [1]:
import pandas as pd
import numpy as np

In [2]:
TRAINING_DATA_PATH = '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_train.csv'
TEST_DATA_PATH = '/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/kumar_processed_with_ID_and_full_perspective_clean_test.csv'

data = pd.concat([
    pd.read_csv(TRAINING_DATA_PATH).drop(columns=['extreme_annotator']),
    pd.read_csv(TEST_DATA_PATH).drop(columns=['extreme_annotator'])
]).reset_index(drop=True)

data

,comment,text_id,worker_id,toxic_score,annotator_id
0,Just a matter of time before pick up on this s...,0,24482c451b411b96d2c2880bafbab9884007e000d143c0...,0,0
1,Just a matter of time before pick up on this s...,0,dbc501198ada6725d8e8cc6f0101824f04d4b4b8935059...,0,1
2,Just a matter of time before pick up on this s...,0,29a3513367445e0fd3c53d61da1fcbebbf4efc6e0de0b9...,0,2
3,Just a matter of time before pick up on this s...,0,26523080557217fc3b42c882aecab5863966ccfbe31c3f...,0,3
4,Just a matter of time before pick up on this s...,0,aa351a28dee4f23fd6abcbb91f9d663440825c8d7455b7...,0,4
...,...,...,...,...,...
534367,My friend was buying a game from gamestop and ...,106030,fd2b94f93b67f7f8a0643ffd5c58e463ed807317ea8174...,1,17106
534368,My friend was buying a game from gamestop and ...,106030,4e3ea78083d10f22c6d3b81d31caec4c6d1adf8486ad79...,1,17107
534369,My friend was buying a game from gamestop and ...,106030,861e81f540c445ccdd16ab6d44e1f0ba45b497d5896583...,1,17108
534370,My friend was buying a game from gamestop and ...,106030,bf743766bfd85b146bb4f1a8de00fad45fe9f6f78c18e4...,0,17109


Analysis (treating the toxicity score as if it was continuous):
1. Compute the average toxicity score for each text.
2. For each (post, annotator) pair, compute the deviation of the score given by that annotator to that post w.r.t. to the post's average score.
3. For each annotator, compute:
    - Number of posts annotatetd.
    - Average score deviation from post mean.
    - Percentage of posts rated as less toxic than the average.
    - Percentage of posts rated as more toxic than the average.
4. Identify the extreme annotators as those who have more than 50% of their posts rated below the post's average.

In [35]:
annotators_data = pd.merge(
    left=data,
    right=(
        data
        .groupby('text_id')['toxic_score']
        .mean()
        .reset_index()
        .rename(columns={'toxic_score': 'mean_toxic_score_text'})
    ),
    on='text_id',
    how='left'
)

annotators_data['toxic_score_deviation'] = annotators_data['toxic_score'] - annotators_data['mean_toxic_score_text']

annotators_data = annotators_data.groupby('annotator_id').agg(
    n_annotated_texts=pd.NamedAgg('text_id', 'nunique'),
    mean_deviation=pd.NamedAgg('toxic_score_deviation', 'mean'),
    percent_lower=pd.NamedAgg('toxic_score_deviation', lambda deviations: np.mean(deviations < -0.5) * 100.)
).reset_index()

annotators_data['extreme_annotator'] = annotators_data['percent_lower'] > 50.

annotators_data

,annotator_id,n_annotated_texts,mean_deviation,percent_lower,extreme_annotator
0,0,40,-0.080000,7.500000,False
1,1,60,-0.061667,5.000000,False
2,2,100,-0.041631,9.000000,False
3,3,60,-0.023333,6.666667,False
4,4,20,-0.010000,0.000000,False
...,...,...,...,...,...
17105,17105,20,-0.130000,25.000000,False
17106,17106,20,0.155000,5.000000,False
17107,17107,20,-0.145000,25.000000,False
17108,17108,20,0.205000,5.000000,False


In [37]:
# annotators_data.to_csv('/data1/moscato/personalised-hate-boundaries-data/data/kumar_perspective_clean/annotators_data.csv', index=None)